# Retrain the model using the complete dataset

TODO: finding the best epoch should not be necessary in this stage. After fixing the tuning notebook, the steps for finding the best epoch in this notebook must be removed.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers, regularizers
import keras_tuner as kt
import sys
import os

# Sets off SettingWithCopyWarning.
pd.set_option('mode.chained_assignment', None)


# ----------------------------------------
# Flags for working on my different machines.
# flag_kaggle = True
flag_FW = True
# flag_LN = True

try:
    if flag_kaggle:
        sys.path.insert(0, '/kaggle/input/hms-lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
        output_dir = ''
except:
    pass 

try:
    if flag_FW:
        sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 

try:
    if flag_LN:
        sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 
# ----------------------------------------

from KLmetric import score

# path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
# path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
# path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
# path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
# path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
# path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

2024-03-14 10:42:08.977181: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 10:42:09.061488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path_train = f'{devset_dir}/05_reduced_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_reduced_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_reduced_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_reduced_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_reduced_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_reduced_single_cwt_v1_test_items.npy'

In [3]:
#
# Data generator for training.
#
# coefficients of cwt's arrays
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y


#
# Test Data generator for predicting
# 

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X


In [4]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 512
Observations in validation set: 128


In [9]:
checkpoint_filepath = f'{output_dir}checkpoint1.model.keras'
model = keras.models.load_model(checkpoint_filepath)


In [11]:
checkpoint_filepath = f'{output_dir}checkpoint1-retrain.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # monitor='val_kl_divergence',  # Name in Kaggle.
    monitor='val_kullback_leibler_divergence',  # Name in FW.
    mode='min',
    save_best_only=True)


In [12]:
history = model.fit(training_generator, epochs=20,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])


Epoch 1/20
16/16 [==============================] - 2s 112ms/step - loss: 1.0570 - kullback_leibler_divergence: 1.0570 - val_loss: 1.3689 - val_kullback_leibler_divergence: 1.3689
Epoch 2/20
16/16 [==============================] - 2s 104ms/step - loss: 0.9184 - kullback_leibler_divergence: 0.9184 - val_loss: 1.4460 - val_kullback_leibler_divergence: 1.4460
Epoch 3/20
16/16 [==============================] - 2s 105ms/step - loss: 0.7506 - kullback_leibler_divergence: 0.7506 - val_loss: 1.4891 - val_kullback_leibler_divergence: 1.4891
Epoch 4/20
16/16 [==============================] - 2s 105ms/step - loss: 0.6915 - kullback_leibler_divergence: 0.6915 - val_loss: 1.6910 - val_kullback_leibler_divergence: 1.6910
Epoch 5/20
16/16 [==============================] - 2s 107ms/step - loss: 0.6402 - kullback_leibler_divergence: 0.6402 - val_loss: 1.7512 - val_kullback_leibler_divergence: 1.7512
Epoch 6/20
16/16 [==============================] - 2s 104ms/step - loss: 0.5835 - kullback_leibler_

In [15]:
val_kl_per_epoch = history.history['val_kullback_leibler_divergence']
best_epoch = val_kl_per_epoch.index(min(val_kl_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

Best epoch: 1


## Merge training and validation sets.

In [27]:
#
# Data generator for training.
#
# coefficients of cwt's arrays
# 5 channels (LP, RP, LT, RP, C)
#

class RetrainDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_train_items, path_to_train_data, path_to_val_items, path_to_val_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.concatenate([np.load(path_to_train_data), np.load(path_to_val_data)])
        self.items = np.concatenate([np.load(path_to_train_items), np.load(path_to_val_items)])
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def get_num_observations(self):
        return self.data.shape[0]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y


In [28]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

retraining_generator = RetrainDataGenerator(path_train_items, path_train, path_val_items, path_val, **params)

print("Observations in training set:", retraining_generator.get_num_observations())


Observations in training set: 600


In [29]:
checkpoint_filepath = f'{output_dir}checkpoint1-retrain.model.keras'
model = keras.models.load_model(checkpoint_filepath)

history = model.fit(retraining_generator, epochs=5)


Epoch 1/5
19/19 [==============================] - 2s 90ms/step - loss: 1.0332 - kullback_leibler_divergence: 1.0332
Epoch 2/5
19/19 [==============================] - 2s 92ms/step - loss: 0.9766 - kullback_leibler_divergence: 0.9766
Epoch 3/5
19/19 [==============================] - 2s 93ms/step - loss: 0.8576 - kullback_leibler_divergence: 0.8576
Epoch 4/5
19/19 [==============================] - 2s 91ms/step - loss: 0.7813 - kullback_leibler_divergence: 0.7813
Epoch 5/5
19/19 [==============================] - 2s 89ms/step - loss: 0.7325 - kullback_leibler_divergence: 0.7325


In [31]:
model_filename = f'{output_dir}hms-keras-12-cwt-reduced.keras'
model.save(model_filename)

In [32]:
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)

y_pred = model.predict(test_generator)

4/4 [==============================] - 0s 21ms/step


In [33]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

test_items = np.load(path_test_items)
df_test_items = pd.DataFrame(test_items)
df_test_items[0] = df_test_items[0].astype(int)

sub = pd.DataFrame({'eeg_id':df_test_items[0]})
sub[TARGETS] = np.round(y_pred,6)
# sub.to_csv('submission.csv',index=False)

df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
df_test_scoring.columns = sub.columns

score(df_test_scoring, sub, 'eeg_id')

1.7986183592759002